# Assignment 3 For Externship program
**Krish Bagga 20BAI1044**

#### Task: Build a CNN model for Bird species

In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential

In [1]:
# unzipping the zip file containing image dataset
!unzip '/content/Externship Dataset/bird_classification.zip'

unzip:  cannot find or open /content/Externship Dataset/bird_classification.zip, /content/Externship Dataset/bird_classification.zip.zip or /content/Externship Dataset/bird_classification.zip.ZIP.


##### Data Augmentation

In [ ]:
# import required libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(
    rescale=(1./255), # (0 to 255) convert to (0 to 1)
    #rotation_range=20,  # Rotate images randomly up to 20 degrees
    #width_shift_range=0.2,  # Shift images horizontally by a fraction of total width
    #height_shift_range=0.2,  # Shift images vertically by a fraction of total height
    shear_range=0.2,  # Shear transformations
    #zoom_range=0.2,  # Randomly zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
)
test_gen = ImageDataGenerator(rescale=(1./255))  

In [ ]:
train = train_gen.flow_from_directory('/content/test_data/test_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/train_data/train_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)

Found 157 images belonging to 16 classes.
Found 150 images belonging to 16 classes.


In [ ]:
# getting indices of class names
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

##### CNN model

In [ ]:
# importing the required libraries
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential

In [ ]:
# Initializing the sequential model
model = Sequential()
# Adding convolutional layer with input
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(120, 120, 3)))
# Normalizing the convolutional layer output
model.add(BatchNormalization())
# Selecting the maximum values
model.add(MaxPooling2D(pool_size=(2,2)))
# Dropping the unwanted 20% of data
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Convolution2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
# Hidden layers
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
# Output layer
model.add(Dense(16,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 118, 118, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 57, 57, 64)       2

In [ ]:
# compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

##### Early stopping

In [ ]:
# import required libraries
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='accuracy',patience=8)

In [ ]:
# fit the model and use early stopping
history = model.fit(train,batch_size=8,validation_data=test,epochs=50,callbacks=early_stop)

19/20 [===========================>..] - ETA: 2s - loss: 1.5115 - accuracy: 0.4832

KeyboardInterrupt: ignored

##### Plotting accuracy and loss

In [ ]:
# import the required libraries
import matplotlib.pyplot as plt

In [ ]:
# Plot training accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy for CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss for CNN')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

##### Testing

In [ ]:
# importing required libraries
from tensorflow.keras.preprocessing import image

# testing on random image
img = image.load_img('/content/test_data/test_data/lbicrw/100_4917.JPG',target_size=(120,120))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred = np.argmax(model.predict(img))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

#### Image classification of bird species using transfer learning (using ResNet 50)

##### ResNet50 model

In [ ]:
# import required libraries
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

In [ ]:
# initialise ResNet50 model with input shape
resnet = ResNet50(include_top=False,input_shape=(120,120,3))

In [ ]:
# keeping the same weights as when trained on imagenet
for layer in resnet.layers:
  layer.trainable=False

In [ ]:
# adding flatten layer and an output dense layer to the ResNet50 model
flatten_layer = Flatten()(resnet.output)
output_layer = Dense(16, activation='softmax')(flatten_layer)
resnet_model = Model(inputs=resnet.input,outputs=output_layer)

In [ ]:
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 126, 126, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 60, 60, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 60, 60, 64)   256         ['conv1_conv[0][0]']         

In [ ]:
# compiling the model
resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# fitting the model on data
resnet_history = resnet_model.fit(train,epochs=30,validation_data=test,callbacks=early_stop)

Epoch 1/30
20/20 [==============================] - 85s 4s/step - loss: 2.4355 - accuracy: 0.1783 - val_loss: 2.6011 - val_accuracy: 0.1733
Epoch 2/30
20/20 [==============================] - 75s 4s/step - loss: 2.5002 - accuracy: 0.2166 - val_loss: 2.6713 - val_accuracy: 0.2333
Epoch 3/30
20/20 [==============================] - 85s 4s/step - loss: 2.4460 - accuracy: 0.2293 - val_loss: 2.6785 - val_accuracy: 0.2400
Epoch 4/30
20/20 [==============================] - 77s 4s/step - loss: 2.3847 - accuracy: 0.2293 - val_loss: 2.5155 - val_accuracy: 0.1400
Epoch 5/30
20/20 [==============================] - 75s 4s/step - loss: 2.2730 - accuracy: 0.3057 - val_loss: 2.4595 - val_accuracy: 0.2800
Epoch 6/30
20/20 [==============================] - 74s 4s/step - loss: 2.1949 - accuracy: 0.3248 - val_loss: 2.7989 - val_accuracy: 0.2000
Epoch 7/30
20/20 [==============================] - 77s 4s/step - loss: 2.2151 - accuracy: 0.3248 - val_loss: 2.3797 - val_accuracy: 0.2467
Epoch 8/30
20/20 [==

##### Plotting accuracy and loss

In [ ]:
# Plot training accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy for ResNet50')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Plot training loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss for ResNet50')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

##### Testing

In [ ]:
# testing on random image
img = image.load_img('/content/test_data/test_data/cbrtsh/_D32_10313.jpg',target_size=(120,120))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred = np.argmax(resnet_model.predict(img))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

In [ ]:
# testing on another random image
img = image.load_img('/content/test_data/test_data/rebimg',target_size=(120,120))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred = np.argmax(resnet_model.predict(img))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])